# Codebook Overview

Here we take the the Box Office Mojo data we extracted and add unique IMDB ids and other features to the movies using the imdb ttitle basics file (Collected from IMDB website). At the end we will have our final fact table.

We are using data from 2009 - 2025. This timeline was selected to capture the complete lifecycle of the modern box office ecosystem, specifically isolating the era defined by the rise of Digital 3D Cinema (beginning with Avatar in late 2009) and the structural disruption of the Streaming Era. The Avatar franchise started in 2009 so it will further help us create features using data from prequels. (Further discussed during feature engineering)

# Creating the Fact Table

## Installing Dependencies

In [5]:
import pandas as pd
import numpy as np
import re
import os
import sys
import subprocess

## Loading the Datasets

The code below automatically loads the data in this notebook. The links are also attached to this cell.

* [dataset1_path](https://drive.google.com/drive/folders/1wz5-HnY_JPsQerZg9fKwxM51cOSQC5t5?dmr=1&ec=wgc-drive-hero-goto) (Box Office Mojo data)
* [dataset2_path](https://drive.google.com/drive/folders/1z2x3i5Z1GFy32ruxHfmnEbveaEmlMqdM?dmr=1&ec=wgc-drive-hero-goto) (IMDB title basics file)

In [6]:
# AUTO-INSTALL GDOWN (If missing)
try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown


# Map the "Filename you want" to the "Google Drive Link"
files = {
    "box_office_data.csv": "https://drive.google.com/file/d/1gQ1gaInO5xqGWGhp1v_gA0hdsN_L_1By/view?usp=drive_link",
    "title.basics.tsv":    "https://drive.google.com/file/d/1xYt8GaBlPcw_QrsQkjb4KiX9LzbANRux/view?usp=drive_link"
}

# 3. DOWNLOADER LOOP
for filename, drive_link in files.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")

        # Extract ID from the link safely
        file_id = drive_link.split('/d/')[1].split('/')[0]
        url = f'https://drive.google.com/uc?id={file_id}'

        # Download (quiet=False shows the progress bar)
        gdown.download(url, filename, quiet=False)
    else:
        print(f"Found {filename} locally.")


# Load Mojo (CSV)
df_Mojo = pd.read_csv("box_office_data.csv", low_memory=False)

# Load IMDB (TSV) - Keeping your tab separator logic
df_IMDB = pd.read_csv("title.basics.tsv", sep='\t',  low_memory=False) # Tab seperator for the TSV file

Found box_office_data.csv locally.
Found title.basics.tsv locally.


In [7]:
df_Mojo.head()

,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,Distributor,date
0,1,1,Marley & Me,"$9,956,212",44.60%,-30.80%,"3,480","$2,860","$82,400,283",8,Twentieth Century Fox,1/1/2009
1,2,2,Bedtime Stories,"$8,336,917",46.40%,-21.20%,"3,681","$2,264","$65,037,829",8,Walt Disney Studios Motion Pictures,1/1/2009
2,3,3,The Curious Case of Benjamin Button,"$7,939,690",85.40%,-33.10%,"2,988","$2,657","$60,605,838",8,Paramount Pictures,1/1/2009
3,4,4,Valkyrie,"$5,747,446",64.80%,-32.30%,"2,711","$2,120","$46,649,304",8,United Artists,1/1/2009
4,5,5,Yes Man,"$5,567,221",102.30%,-6.10%,"3,434","$1,621","$65,596,911",14,Warner Bros.,1/1/2009


In [8]:
df_Mojo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269249 entries, 0 to 269248
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   TD           269249 non-null  int64 
 1   YD           269249 non-null  object
 2   Release      269249 non-null  object
 3   Daily        269249 non-null  object
 4   %± YD        269249 non-null  object
 5   %± LW        269249 non-null  object
 6   Theaters     269249 non-null  object
 7   Avg          269249 non-null  object
 8   To Date      269249 non-null  object
 9   Days         269249 non-null  object
 10  Distributor  268447 non-null  object
 11  date         269249 non-null  object
dtypes: int64(1), object(11)
memory usage: 24.7+ MB


In [9]:
df_Mojo.columns

Index(['TD', 'YD', 'Release', 'Daily', '%± YD', '%± LW', 'Theaters', 'Avg',
       'To Date', 'Days', 'Distributor', 'date'],
      dtype='object')

### Column Details (Box Office Mojo)

* TD (Today's Rank): The movie's ranking for the specific day based on daily gross revenue (e.g., 1 means it was the top-grossing movie that day).

* YD (Yesterday's Rank): The movie's ranking on the previous day.

* Release: The title of the movie.

* Daily: The total box office revenue (gross) generated by the movie on that specific date.

* %± YD (Percent Change from Yesterday): The percentage increase or decrease in revenue compared to the previous day.

* %± LW (Percent Change from Last Week): The percentage increase or decrease in revenue compared to the same day one week prior. This is a key metric for measuring a movie's "holding power."

* Theaters: The total number of movie theaters showing the film on that date.

* Avg (Average per Theater): The Daily gross divided by the number of Theaters. This indicates how crowded the screenings were on average.

* To Date: The cumulative total gross revenue earned by the movie from its release up to the current date.

* Days: The number of days the movie has been in release (e.g., Day 1 is opening day).

* Distributor: The studio or company responsible for releasing the movie (e.g., Disney, Warner Bros., A24).

* date: The specific calendar date for this row of data.

In [10]:
df_IMDB.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


In [11]:
df_IMDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12058793 entries, 0 to 12058792
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         int64 
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: int64(1), object(8)
memory usage: 828.0+ MB


In [12]:
df_IMDB.columns

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')

### Column Details (IMDB title basics)

* tconst: The unique alphanumeric identifier for the title (e.g., tt0000001). This is the primary key used to link this movie to other IMDb datasets (like ratings, crew, etc.).

* titleType: The format or type of the title. Common values include movie, short, tvSeries, tvEpisode, video, etc.

* primaryTitle: The most popular title used for promotional materials or the title by which it is known in your selected region (usually English). This is the one we should use for matching with Box Office Mojo.

* originalTitle: The title in the original language. For international films, this might differ from the primaryTitle.

* isAdult: A flag indicating if the title is an adult film.

* startYear: The release year of the title. For TV series, this is the year the series started.

* endYear: The year a TV series ended. For movies or ongoing series, this will be null (\N).

* runtimeMinutes: The primary runtime of the title in minutes.

* genres: A comma-separated list of up to three genres associated with the title (e.g., Action,Adventure,Sci-Fi).

**We filter the IMDB title basics file and keep only keep the movie data as the rest is not relevant for our model**

In [13]:
df_IMDB = df_IMDB[df_IMDB['titleType'] == 'movie'].copy()

In [14]:
df_IMDB

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
331,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,40,"Biography,Drama"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
...,...,...,...,...,...,...,...,...,...
12058687,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,\N,57,Documentary
12058711,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,\N,100,Documentary
12058723,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,\N,\N,Comedy
12058733,tt9916730,movie,6 Gunn,6 Gunn,0,2017,\N,116,Drama


## Merging the 2 Files

### Box Office Mojo Data Preparation

* We will merge the two datasets using their titles and release years. To standardize the titles we first create a title cleaning function using regex to remove all special characters and spaces from the title name. We create merge_key_title using this.

* We also create merge_key_year by extracting year data from the date column.

In [15]:
# 1. Define the title-cleaning function
def clean_title(title):
    if not isinstance(title, str):
        return ""
    title = title.lower()
    title = re.sub(r'[^a-z0-9]', '', title) # Cleans to 'avatarthewayofwater'
    return title


# 2. Create Merge Keys
df_Mojo['merge_key_title'] = df_Mojo['Release'].apply(clean_title)
df_Mojo['date'] = pd.to_datetime(df_Mojo['date'], errors='coerce')
df_Mojo['merge_key_year'] = df_Mojo['date'].dt.year

# 3. Drop rows with bad data
df_Mojo = df_Mojo.dropna(subset=['date', 'Daily', 'merge_key_title', 'merge_key_year'])
df_Mojo['merge_key_year'] = df_Mojo['merge_key_year'].astype(int)

# We enforce the *true* primary key of this table: one entry per movie, per day.
df_Mojo = df_Mojo.drop_duplicates(
    subset=['Release', 'date'], # The real unique key
    keep='first'
).copy()

df_Mojo.head()

,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,Distributor,date,merge_key_title,merge_key_year
0,1,1,Marley & Me,"$9,956,212",44.60%,-30.80%,"3,480","$2,860","$82,400,283",8,Twentieth Century Fox,2009-01-01,marleyme,2009
1,2,2,Bedtime Stories,"$8,336,917",46.40%,-21.20%,"3,681","$2,264","$65,037,829",8,Walt Disney Studios Motion Pictures,2009-01-01,bedtimestories,2009
2,3,3,The Curious Case of Benjamin Button,"$7,939,690",85.40%,-33.10%,"2,988","$2,657","$60,605,838",8,Paramount Pictures,2009-01-01,thecuriouscaseofbenjaminbutton,2009
3,4,4,Valkyrie,"$5,747,446",64.80%,-32.30%,"2,711","$2,120","$46,649,304",8,United Artists,2009-01-01,valkyrie,2009
4,5,5,Yes Man,"$5,567,221",102.30%,-6.10%,"3,434","$1,621","$65,596,911",14,Warner Bros.,2009-01-01,yesman,2009


In [16]:
df_Mojo.shape

(269249, 14)

### IMDB Title Basics Data Preparation

* We apply the title cleaing function here as well to create merge_key_title.

* We also extract the release year as we will ber merging on title and year.

In [17]:
#  1. Create Merge Keys
df_IMDB_clean = df_IMDB.copy()
df_IMDB_clean['merge_key_title'] = df_IMDB_clean['primaryTitle'].apply(clean_title)
df_IMDB_clean['imdb_year'] = pd.to_numeric(df_IMDB_clean['startYear'], errors='coerce')

# Drop rows where we can't create a valid key (missing title or year)
df_IMDB_clean = df_IMDB_clean.dropna(subset=['merge_key_title', 'imdb_year'])
df_IMDB_clean['imdb_year'] = df_IMDB_clean['imdb_year'].astype(int)

#  2. De-duplicate the bridge
# We only want one IMDb entry for each 'title + year' combo for the lookup process
df_IMDB_clean = df_IMDB_clean.drop_duplicates(
    subset=['merge_key_title', 'imdb_year'],
    keep='first'
).copy()

df_IMDB_clean.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,merge_key_title,imdb_year
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,missjerry,1894
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport",thecorbettfitzsimmonsfight,1897
331,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,40,"Biography,Drama",soldiersofthecross,1900
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,bohemios,1905
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",thestoryofthekellygang,1906


In [18]:
df_IMDB_clean.shape

(617061, 11)

### 4 Year Tolerance

Instead of just searching for the exact title once, the code runs a series of searches with widening criteria:

* Pass 1 (Strict): Search for "{Title} {Year} film" (e.g., "Avatar 2009 film").

* Pass 2 (Tolerance): If that fails, search for the title with years +/- 2 years from the listed release date (e.g., "Avatar 2008 film", "Avatar 2010 film").

**Why We Did It**

* Festival vs. Theatrical Dates: Databases often disagree on the "Release Year". IMDb might list 2023 (when a movie premiered at a film festival) but Mojo dates begin when the film was premiered in the domestic box office.

* International Release Lags: Movies often release in different markets in different years (e.g., December 2023 in the US, January 2024 in the UK).

**This ultimately helps us get a comprehensive dataset with the maximum amount of possible matches to have a comprehensive fact table**

In [19]:
print("Creating the 'Rosetta Stone' (lookup file)...")

# 1. Get unique list of movies
box_office_movies = df_Mojo[['Release', 'merge_key_title', 'merge_key_year']].drop_duplicates().copy()
print(f"Found {len(box_office_movies)} unique movies in your Box Office data.")

# 2. Prepare Lookup Data
imdb_simple = df_IMDB_clean[['merge_key_title', 'imdb_year', 'tconst']].copy()

# 3. Pass 1: Exact Match
print("Running Pass 1: Exact Match...")
df_rosetta = pd.merge(
    box_office_movies,
    imdb_simple,
    left_on=['merge_key_title', 'merge_key_year'],
    right_on=['merge_key_title', 'imdb_year'],
    how='left'
)

# 4. Pass 2: Tolerance Loop
offsets = [-1, 1, -2, 2, -3, 3, -4, 4]

for offset in offsets:
    # Check what is still missing
    missing_mask = df_rosetta['tconst'].isna()
    if not missing_mask.any(): break

    # Prepare missing rows
    df_missing = df_rosetta[missing_mask].copy()

    # CRITICAL FIX: Save the original index to map back correctly
    df_missing['original_index'] = df_missing.index

    df_missing['target_year'] = df_missing['merge_key_year'] + offset

    # Attempt merge
    df_match_attempt = pd.merge(
        df_missing.drop(columns=['tconst', 'imdb_year'], errors='ignore'),
        imdb_simple,
        left_on=['merge_key_title', 'target_year'],
        right_on=['merge_key_title', 'imdb_year'],
        how='left'
    )

    # Update matches
    # We filter for success, then use the SAVED 'original_index' to write back
    matches = df_match_attempt[df_match_attempt['tconst'].notna()]

    if not matches.empty:
        print(f"  -> Offset {offset:+d} years: Found {len(matches)} matches.")

        # FIX: Write data to the *original* locations, not the new merge locations
        # We use .values to ignore the match's new index and purely align by position
        df_rosetta.loc[matches['original_index'], 'tconst'] = matches['tconst'].values

print(f"--- Multi-Pass Complete. Matches so far: {df_rosetta['tconst'].notna().sum()} ---")

Creating the 'Rosetta Stone' (lookup file)...
Found 6717 unique movies in your Box Office data.
Running Pass 1: Exact Match...
  -> Offset -1 years: Found 1860 matches.
  -> Offset +1 years: Found 10 matches.
  -> Offset -2 years: Found 179 matches.
  -> Offset +2 years: Found 3 matches.
  -> Offset -3 years: Found 24 matches.
  -> Offset -4 years: Found 5 matches.
  -> Offset +4 years: Found 4 matches.
--- Multi-Pass Complete. Matches so far: 6026 ---


### Final Merge

In [20]:
print(f"Constructing Final Master Dataset...")
print(f"Original Daily Records: {len(df_Mojo)}")

#  1. Merge Daily Data with the Rosetta Stone
# This attaches the 'tconst' (ID) to every daily box office record.
# We use a LEFT merge to keep every single box office record.
df_master = pd.merge(
    df_Mojo,
    df_rosetta[['Release', 'merge_key_title', 'merge_key_year', 'tconst']],
    on=['Release', 'merge_key_title', 'merge_key_year'],
    how='left'
)

#  2. Merge IMDb Features
# We use the FULL IMDb dataset now to get genres, runtime, etc.
# CRITICAL: Drop duplicates on 'tconst' first to prevent row explosion.
# (IMDb sometimes has duplicate IDs for different regions; we only want one row of features per movie)
df_imdb_features = df_IMDB.drop_duplicates(subset=['tconst'])

df_master = pd.merge(
    df_master,
    df_imdb_features,
    on='tconst',
    how='left',
    suffixes=('_mojo', '_imdb') # Handle any colliding column names
)

#  3. Validation
print("-" * 30)
# Check if row count changed (It shouldn't if unique keys were handled correctly)
if len(df_master) == len(df_Mojo):
    print("✅ SUCCESS: Row count preserved.")
else:
    print(f"⚠️ WARNING: Row count changed! Original: {len(df_Mojo)}, New: {len(df_master)}")
    print("Likely cause: Duplicates in df_rosetta or df_IMDB.")

# Stats
matched_count = df_master['tconst'].notna().sum()
print(f"Final Shape: {df_master.shape}")
print(f"Rows with enriched IMDb Data: {matched_count} ({matched_count / len(df_master):.1%})")

#  4. Save to CSV
save_path = 'final_merged_box_office.csv'
print(f"Saving to {save_path}...")
df_master.to_csv(save_path, index=False)
print("✅ File saved successfully!")

# Preview
df_master.head()

Constructing Final Master Dataset...
Original Daily Records: 269249
------------------------------
✅ SUCCESS: Row count preserved.
Final Shape: (269249, 23)
Rows with enriched IMDb Data: 256013 (95.1%)
Saving to final_merged_box_office.csv...
✅ File saved successfully!


,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,...,merge_key_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,1,1,Marley & Me,"$9,956,212",44.60%,-30.80%,"3,480","$2,860","$82,400,283",8,...,2009,tt0822832,movie,Marley & Me,Marley & Me,0.0,2008,\N,115,"Drama,Family"
1,2,2,Bedtime Stories,"$8,336,917",46.40%,-21.20%,"3,681","$2,264","$65,037,829",8,...,2009,tt0960731,movie,Bedtime Stories,Bedtime Stories,0.0,2008,\N,99,"Adventure,Comedy,Family"
2,3,3,The Curious Case of Benjamin Button,"$7,939,690",85.40%,-33.10%,"2,988","$2,657","$60,605,838",8,...,2009,tt0421715,movie,The Curious Case of Benjamin Button,The Curious Case of Benjamin Button,0.0,2008,\N,166,"Drama,Fantasy,Romance"
3,4,4,Valkyrie,"$5,747,446",64.80%,-32.30%,"2,711","$2,120","$46,649,304",8,...,2009,tt0985699,movie,Valkyrie,Valkyrie,0.0,2008,\N,121,"Drama,History,Thriller"
4,5,5,Yes Man,"$5,567,221",102.30%,-6.10%,"3,434","$1,621","$65,596,911",14,...,2009,tt1068680,movie,Yes Man,Yes Man,0.0,2008,\N,104,"Comedy,Romance"


In [21]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269249 entries, 0 to 269248
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   TD               269249 non-null  int64         
 1   YD               269249 non-null  object        
 2   Release          269249 non-null  object        
 3   Daily            269249 non-null  object        
 4   %± YD            269249 non-null  object        
 5   %± LW            269249 non-null  object        
 6   Theaters         269249 non-null  object        
 7   Avg              269249 non-null  object        
 8   To Date          269249 non-null  object        
 9   Days             269249 non-null  object        
 10  Distributor      268447 non-null  object        
 11  date             269249 non-null  datetime64[ns]
 12  merge_key_title  269249 non-null  object        
 13  merge_key_year   269249 non-null  int64         
 14  tconst           256

While most of the merge was successful, we can see that there are a bunch of null values in the IMDB data information. We will now take a look at why that may be the case.

In [22]:
# These are the rows from Box Office Mojo that found no match in IMDb
unmerged_df = df_master[df_master['tconst'].isnull()].copy()

print(f"Total Unmerged Rows: {len(unmerged_df)}")

# Define Keywords for Special Runs
# These are common terms Box Office Mojo adds to titles for non-standard releases
special_keywords = [
    're-release',
    'rerelease',
    'special edition',
    'imax',
    '3d',
    '4k',
    'anniversary',
    'double feature',
    'marathon',
    'sing-along',
    'extended'
]

# Create a regex pattern to search for ANY of these words
pattern = '|'.join(special_keywords)

# Filter for Special Runs
# We check the 'Release' column (the name from Box Office Mojo)
special_run_rows = unmerged_df[unmerged_df['Release'].str.contains(pattern, case=False, na=False)]

# Calculate Statistics
special_count = len(special_run_rows)
total_unmerged = len(unmerged_df)
percent_special = (special_count / total_unmerged) * 100

print(f"Rows identified as Special Runs/Re-releases: {special_count}")
print(f"Percentage of unmerged data that is a special run: {percent_special:.2f}%")

# Inspect Samples
print("\n--- Sample of Identified Special Runs ---")
print(special_run_rows['Release'].unique()[:10])

print("\n--- Sample of other Unmerged Rows ---")
others = unmerged_df[~unmerged_df.index.isin(special_run_rows.index)]
print(others['Release'].unique()[:10])

Total Unmerged Rows: 13236
Rows identified as Special Runs/Re-releases: 8733
Percentage of unmerged data that is a special run: 65.98%

--- Sample of Identified Special Runs ---
['The Polar Express2008 IMAX Release' 'Under the Sea 3D'
 'Toy Story2009 Re-release' 'Bicycle Thieves2008 Re-release'
 'The Polar Express2009 IMAX Release' '44 Inch Chest2010 Re-release'
 'The Square2010 Re-release' 'Splice2010 Re-release'
 "Gangster's Paradise: Jerusalema2010 Re-release"
 'Breathless2010 Re-release']

--- Sample of other Unmerged Rows ---
['Deep Sea' 'Lola Montès' 'Che: Part One/Che: Part Two' 'Z'
 'Léon Morin, Priest' 'The Wizard of Oz2009 NCM Fathom Event'
 'KitesThe Remix' 'Knight and DayOriginal Release'
 'Twilight/New MoonDouble Bill' 'Grease2010 Sing-a-long']


From the check it is evident that most of the unmerged rows represent special runs or re runs which we can safely drop (As our model needs to predict performance after the original releases only)

### Sanity Checks
* We now have our raw fact table. We will clean it up after running some sanity checks.

In [23]:
# Filter by Title string and Release Year
df_avatar_first = df_master[
    (df_master['Release'] == 'Avatar') &
    (df_master['merge_key_year'] == 2009)
].copy()

# Sort by date to see the run progression
df_avatar_first = df_avatar_first.sort_values('date')

df_avatar_first.head()

,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,...,merge_key_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
16820,1,-,Avatar,"$26,752,099",-,-,"3,452","$7,749","$26,752,099",1,...,2009,tt0499549,movie,Avatar,Avatar,0.0,2009,\N,162,"Action,Adventure,Fantasy"
16868,1,1,Avatar,"$25,529,036",-4.60%,-,"3,452","$7,395","$52,281,135",2,...,2009,tt0499549,movie,Avatar,Avatar,0.0,2009,\N,162,"Action,Adventure,Fantasy"
16916,1,1,Avatar,"$24,744,346",-3.10%,-,"3,452","$7,168","$77,025,481",3,...,2009,tt0499549,movie,Avatar,Avatar,0.0,2009,\N,162,"Action,Adventure,Fantasy"
16963,1,1,Avatar,"$16,385,820",-33.80%,-,"3,452","$4,746","$93,411,301",4,...,2009,tt0499549,movie,Avatar,Avatar,0.0,2009,\N,162,"Action,Adventure,Fantasy"
17007,1,1,Avatar,"$16,086,461",-1.80%,-,"3,452","$4,660","$109,497,762",5,...,2009,tt0499549,movie,Avatar,Avatar,0.0,2009,\N,162,"Action,Adventure,Fantasy"


We see the correct information for the first Avatar movie

In [24]:
# The Avengers (2012)
df_avengers_1 = df_master[
    (df_master['Release'] == 'The Avengers') &
    (df_master['merge_key_year'] == 2012)
].copy()

df_avengers_1 = df_avengers_1.sort_values('date')

df_avengers_1.head()

,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,...,merge_key_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
57302,1,-,The Avengers,"$80,813,985",-,-,"4,349","$18,582","$80,813,985",1,...,2012,tt0848228,movie,The Avengers,The Avengers,0.0,2012,\N,143,"Action,Sci-Fi"
57356,1,1,The Avengers,"$69,557,990",-13.90%,-,"4,349","$15,994","$150,371,975",2,...,2012,tt0848228,movie,The Avengers,The Avengers,0.0,2012,\N,143,"Action,Sci-Fi"
57410,1,1,The Avengers,"$57,066,733",-18%,-,"4,349","$13,121","$207,438,708",3,...,2012,tt0848228,movie,The Avengers,The Avengers,0.0,2012,\N,143,"Action,Sci-Fi"
57465,1,1,The Avengers,"$18,898,999",-66.90%,-,"4,349","$4,345","$226,337,707",4,...,2012,tt0848228,movie,The Avengers,The Avengers,0.0,2012,\N,143,"Action,Sci-Fi"
57516,1,1,The Avengers,"$17,677,190",-6.50%,-,"4,349","$4,064","$244,014,897",5,...,2012,tt0848228,movie,The Avengers,The Avengers,0.0,2012,\N,143,"Action,Sci-Fi"


The data for The Avengers movie is also accurate

In [25]:
# Avengers: Infinity War (2018)
df_infinity_war = df_master[
    (df_master['Release'] == 'Avengers: Infinity War') &
    (df_master['merge_key_year'] == 2018)
].copy()

df_infinity_war = df_infinity_war.sort_values('date')

df_infinity_war.head()

,TD,YD,Release,Daily,%± YD,%± LW,Theaters,Avg,To Date,Days,...,merge_key_year,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
165203,1,-,Avengers: Infinity War,"$106,334,939",-,-,"4,474","$23,767","$106,334,939",1,...,2018,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0.0,2018,\N,149,"Action,Adventure,Sci-Fi"
165258,1,1,Avengers: Infinity War,"$82,131,612",-22.80%,-,"4,474","$18,357","$188,466,551",2,...,2018,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0.0,2018,\N,149,"Action,Adventure,Sci-Fi"
165313,1,1,Avengers: Infinity War,"$69,231,632",-15.70%,-,"4,474","$15,474","$257,698,183",3,...,2018,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0.0,2018,\N,149,"Action,Adventure,Sci-Fi"
165368,1,1,Avengers: Infinity War,"$24,740,117",-64.30%,-,"4,474","$5,529","$282,438,300",4,...,2018,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0.0,2018,\N,149,"Action,Adventure,Sci-Fi"
165422,1,1,Avengers: Infinity War,"$23,426,108",-5.30%,-,"4,474","$5,236","$305,864,408",5,...,2018,tt4154756,movie,Avengers: Infinity War,Avengers: Infinity War,0.0,2018,\N,149,"Action,Adventure,Sci-Fi"


Lastly, we also find accurate data about the Avengers: Infinity War.

**The merge passes sanity checks and was done correctly**

# Cleaning the Fact Table

* Now we will only keep the datapoints and columns necessary for our fact table.

## Dropping the Null Values (Includes re-runs and special runs)

In [26]:
# Columns we currently have

df_master.columns

Index(['TD', 'YD', 'Release', 'Daily', '%± YD', '%± LW', 'Theaters', 'Avg',
       'To Date', 'Days', 'Distributor', 'date', 'merge_key_title',
       'merge_key_year', 'tconst', 'titleType', 'primaryTitle',
       'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes',
       'genres'],
      dtype='object')

In [27]:
subset_cols = ['tconst', 'titleType', 'primaryTitle',
       'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes',
       'genres']
df_master.dropna(subset=subset_cols, inplace=True)

We replace the few null values in the distributor column with 'Unknown'

In [28]:
# Replace NaN values in 'Distributor' with 'Unknown'
df_master['Distributor'] = df_master['Distributor'].fillna('Unknown')

## Dropping Columns We Do Not Need

**We drop the following columns as we only want to use historical data to predict performance and not actual performance data after release**
* TD (Today's Rank): The movie's ranking for the specific day based on daily gross revenue (e.g., 1 means it was the top-grossing movie that day).

* YD (Yesterday's Rank): The movie's ranking on the previous day. This helps track whether a movie is climbing or falling in popularity.

* %± YD (Percent Change from Yesterday): The percentage increase or decrease in revenue compared to the previous day.

* %± LW (Percent Change from Last Week): The percentage increase or decrease in revenue compared to the same day one week prior.

**We drop the following column because we already filtered to keep movies only**
* titleType: The format or type of the title. Common values include movie, short, tvSeries, tvEpisode, video, etc.

**Merge keys**
We don't need these as the merge is already done
* merge_key_title
* merge_key_year

**Other irrelevant columns**

endYear: The year a TV series ended.

originalTitle: The title in the original language. For international films, this might differ from the primaryTitle. We are working with domestic release data only so this is not relevant for now as going forward we will be using unique imdb id to merge our data.

In [29]:
# Dropping irrelevant columns

# List of columns to drop based on my criteria
cols_to_drop = [
    'TD',
    'YD',
    '%± YD',
    '%± LW',
    'Days',
    'titleType',
    'merge_key_title',
    'merge_key_year',
    'endYear',
    'originalTitle'
]

df_master.drop(columns=cols_to_drop, inplace=True, errors='ignore')


We reset the index as well for our final fact table

In [30]:
df_master.reset_index(drop=True, inplace=True)

In [31]:
df_master.shape

(256013, 13)

In [32]:
df_master.head()

,Release,Daily,Theaters,Avg,To Date,Distributor,date,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres
0,Marley & Me,"$9,956,212","3,480","$2,860","$82,400,283",Twentieth Century Fox,2009-01-01,tt0822832,Marley & Me,0.0,2008,115,"Drama,Family"
1,Bedtime Stories,"$8,336,917","3,681","$2,264","$65,037,829",Walt Disney Studios Motion Pictures,2009-01-01,tt0960731,Bedtime Stories,0.0,2008,99,"Adventure,Comedy,Family"
2,The Curious Case of Benjamin Button,"$7,939,690","2,988","$2,657","$60,605,838",Paramount Pictures,2009-01-01,tt0421715,The Curious Case of Benjamin Button,0.0,2008,166,"Drama,Fantasy,Romance"
3,Valkyrie,"$5,747,446","2,711","$2,120","$46,649,304",United Artists,2009-01-01,tt0985699,Valkyrie,0.0,2008,121,"Drama,History,Thriller"
4,Yes Man,"$5,567,221","3,434","$1,621","$65,596,911",Warner Bros.,2009-01-01,tt1068680,Yes Man,0.0,2008,104,"Comedy,Romance"


In [33]:
df_master.tail()

,Release,Daily,Theaters,Avg,To Date,Distributor,date,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres
256008,The Conjuring: Last Rites,"$9,318",216,$43,"$177,703,202",Warner Bros.,2025-11-12,tt22898462,The Conjuring: Last Rites,0.0,2025,135,"Horror,Mystery,Thriller"
256009,Soul on Fire,"$8,768",275,$31,"$7,381,918",Affirm Films,2025-11-12,tt28078628,Soul on Fire,0.0,2025,112,"Biography,Drama"
256010,The Bad Guys 2,"$4,860",109,$44,"$82,590,470",Universal Pictures,2025-11-12,tt30017619,The Bad Guys 2,0.0,2025,104,"Action,Adventure,Animation"
256011,Gabby's Dollhouse: The Movie,"$4,195",220,$19,"$31,928,805",Universal Pictures International (UPI),2025-11-12,tt32214143,Gabby's Dollhouse: The Movie,0.0,2025,98,"Adventure,Animation,Comedy"
256012,Belén,$313,10,$31,"$2,317",Amazon MGM Studios,2025-11-12,tt37531036,Belén,0.0,2025,105,"Crime,Drama"


In [34]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256013 entries, 0 to 256012
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Release         256013 non-null  object        
 1   Daily           256013 non-null  object        
 2   Theaters        256013 non-null  object        
 3   Avg             256013 non-null  object        
 4   To Date         256013 non-null  object        
 5   Distributor     256013 non-null  object        
 6   date            256013 non-null  datetime64[ns]
 7   tconst          256013 non-null  object        
 8   primaryTitle    256013 non-null  object        
 9   isAdult         256013 non-null  float64       
 10  startYear       256013 non-null  object        
 11  runtimeMinutes  256013 non-null  object        
 12  genres          256013 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(11)
memory usage: 25.4+ MB


## Datatype Transformation

In [35]:
# Create a copy to be safe
df_clean = df_master.copy()


# Clean Currency/Number Strings (Remove '$' and ',')
cols_to_clean = ['Daily', 'Theaters', 'Avg', 'To Date']

for col in cols_to_clean:
    # Force to string, remove symbols, convert to numeric
    # errors='coerce' turns non-numbers (like '-') into NaN (0)
    df_clean[col] = pd.to_numeric(
        df_clean[col].astype(str).str.replace(r'[$,]', '', regex=True),
        errors='coerce'
    ).fillna(0) # Assume 0 if data is bad/missing

# Clean IMDb Numerics
# IMDb uses '\\N' for missing data, so we coerce errors
df_clean['runtimeMinutes'] = pd.to_numeric(df_clean['runtimeMinutes'], errors='coerce')
df_clean['startYear'] = pd.to_numeric(df_clean['startYear'], errors='coerce')

# Further transformation
df_clean['isAdult'] = df_clean['isAdult'].fillna(0).astype(int)
df_clean['startYear'] = df_clean['startYear'].fillna(0).astype(int)

# Check the new types
print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256013 entries, 0 to 256012
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Release         256013 non-null  object        
 1   Daily           256013 non-null  int64         
 2   Theaters        256013 non-null  float64       
 3   Avg             256013 non-null  float64       
 4   To Date         256013 non-null  float64       
 5   Distributor     256013 non-null  object        
 6   date            256013 non-null  datetime64[ns]
 7   tconst          256013 non-null  object        
 8   primaryTitle    256013 non-null  object        
 9   isAdult         256013 non-null  int64         
 10  startYear       256013 non-null  int64         
 11  runtimeMinutes  254573 non-null  float64       
 12  genres          256013 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 25.4+ MB
None


In [36]:
df_clean.tail()

,Release,Daily,Theaters,Avg,To Date,Distributor,date,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres
256008,The Conjuring: Last Rites,9318,216.0,43.0,177703202.0,Warner Bros.,2025-11-12,tt22898462,The Conjuring: Last Rites,0,2025,135.0,"Horror,Mystery,Thriller"
256009,Soul on Fire,8768,275.0,31.0,7381918.0,Affirm Films,2025-11-12,tt28078628,Soul on Fire,0,2025,112.0,"Biography,Drama"
256010,The Bad Guys 2,4860,109.0,44.0,82590470.0,Universal Pictures,2025-11-12,tt30017619,The Bad Guys 2,0,2025,104.0,"Action,Adventure,Animation"
256011,Gabby's Dollhouse: The Movie,4195,220.0,19.0,31928805.0,Universal Pictures International (UPI),2025-11-12,tt32214143,Gabby's Dollhouse: The Movie,0,2025,98.0,"Adventure,Animation,Comedy"
256012,Belén,313,10.0,31.0,2317.0,Amazon MGM Studios,2025-11-12,tt37531036,Belén,0,2025,105.0,"Crime,Drama"


In [37]:
df_clean.to_csv('fact_table.csv', index=False)

Here, we have our Fact Table which we will aggregate in the next notebook and prepare it for modeling. I have also attached a drive link to the fact table below:

[Fact Table](https://drive.google.com/file/d/19kPMjHRC7nlUrl41NPE01nuN-LqCgP1k/view?usp=drive_link)

# Fact Table Overview

**Identifiers and keys**

* tconst (**Our main identifier**):	The Primary Key. We can use this to link to other tables (Cast, Crew, Reviews).
* Release:	The Box Office Mojo title.
* primaryTitle:	The IMDb title. Good for sanity checking against Release.

**Time dimensions**

* date: The Observation Date. The specific calendar day for this revenue record.
* startYear: "The year the movie was released accordind to IMDB

**Performance metrics**

* Daily: The gross revenue earned only on that specific date.
* To Date: Cumulative Gross. Total revenue earned from the premiere up to this specific date. Indicates momentum.
* Avg: Per-Theater Average. Calculated as Daily / Theaters.

**Distribution Features**

* Theaters: The number of theater locations showing the film on that day.
* Distributor: The studio responsible for releasing the film (e.g., Disney, Warner Bros). Crucial for categorical analysis.

**Content features**

* runtimeMinutes:	The length of the movie.
* genres:	Comma-separated list of genres (e.g., "Action,Adventure").
* isAdult	Binary flag (0 = No, 1 = Yes) for adult content.

We will add features and data from other sources after aggregating our fact table in the next codebook.